In [1]:
# General imports
import cv2
import mediapipe as mp
import numpy as np
import os
import sys
from datetime import datetime, date
import time
import uuid
import seaborn as sns
import matplotlib.pyplot as plt
import pprint


# data processing and visualization helper functions
from processing import process_image, process_video
from data_acquisition import capture_session
from visualization import plot_ROM_dtw_comparisons, calc_DTW_joints_metrics, create_3D_keypoints_video, plot_ex_animation


# Request handling and data payload packaging
from django.core.serializers.json import DjangoJSONEncoder
import json
import requests

# DTW imports
from dtw import *

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



# Per Exercise DTW Testing

In [2]:
EXERCISE = 'LEG_RAISE'
OUTPUT_FOLDER = './captures/'

### Create Reference

In [4]:
date_str = datetime.now().isoformat()
name = f"{EXERCISE}_reference_{date_str}"
date = datetime(2022, 12, 1)
target_reps = 4

# Capture video recording and image
image_path, video_path = capture_session(name, 10, 12, base_dir='./captures/')

Entering Video Loop


In [5]:
# Process image and video
flag, image_output = process_image(image_path)
if not flag:
    assert("Error")

flag, video_output = process_video(video_path)
if not flag:
    assert("Error")

# Prep payload
payload = dict(
    exercise=EXERCISE,
    target_reps= target_reps,
    name = name,
    date=date,
    pose_keypoints = image_output,
    keypoints_sequence = video_output
)

ref_url = 'http://127.0.0.1:8000/api/v1/reference_sessions/'

# Send requests to REST API Endpoint
r = requests.post(ref_url, data=json.dumps(payload, cls=DjangoJSONEncoder), headers={'Content-Type': 'application/json'})
r.raise_for_status()

reference_data = r.json()

print(reference_data['id'])

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


de0e590d-e485-43ad-8034-24094d569345


# Send Test Exercise Sessions

In [6]:
def execute_exercise_session(exercise, name, ref_id, target_reps, image_path, video_path):
    date = datetime(2022, 12, 1)
    
    # Process image and video
    flag, image_output = process_image(image_path)
    if not flag:
        assert("Error")

    flag, video_output = process_video(video_path)
    if not flag:
        assert("Error")

    # Prep payload
    payload = dict(
        exercise=exercise,
        target_reps= target_reps,
        sets=1,
        name = name,
        date=date,
        reference=ref_id,
        pose_keypoints = image_output,
        keypoints_sequence = video_output
    )

    ex_url = 'http://127.0.0.1:8000/api/v1/exercise_sessions/'

    # Send requests to REST API Endpoint
    r = requests.post(ex_url, data=json.dumps(payload, cls=DjangoJSONEncoder), headers={'Content-Type': 'application/json'})
    r.raise_for_status()

    return r.json()


Initialize list to hold pairs of ex and ref data

In [7]:
pairs = []
ref_id = 'de0e590d-e485-43ad-8034-24094d569345'
target_reps = 4

### Session 1: Similar to Reference

In [8]:
date_str = datetime.now().isoformat()
name = f"{EXERCISE}_{'base_session'}_{date_str}"
target_reps = 4

# Capture video recording and image
image_path, video_path = capture_session(name,10,12, base_dir='./captures/')

Entering Video Loop


In [9]:
ex_data = execute_exercise_session(EXERCISE, name, ref_id, target_reps, image_path, video_path)
# pairs.append((ex_data, reference_data))

### Session 2: Diff Angle from Session 1


In [10]:
date_str = datetime.now().isoformat()
name = f"{EXERCISE}_{'diff_angle'}_{date_str}"
target_reps = 4

# Capture video recording and image
image_path, video_path = capture_session(name,10,12, base_dir='./captures/')

Entering Video Loop


In [11]:
ex_data = execute_exercise_session(EXERCISE, name, ref_id, target_reps, image_path, video_path)
# pairs.append((ex_data, reference_data))

### Session 3: Static Pose

In [13]:
date_str = datetime.now().isoformat()
name = f"{EXERCISE}_{'static_pose'}_{date_str}"
target_reps = 4

# Capture video recording and image
image_path, video_path = capture_session(name,10,12, base_dir='./captures/')

Entering Video Loop


In [14]:
ex_data = execute_exercise_session(EXERCISE, name, ref_id, target_reps, image_path, video_path)
# pairs.append((ex_data, reference_data))

### Session 4: Pauses

In [16]:
date_str = datetime.now().isoformat()
name = f"{EXERCISE}_{'pauses'}_{date_str}"
target_reps = 4

# Capture video recording and image
image_path, video_path = capture_session(name,10,20, base_dir='./captures/')

Entering Video Loop


In [17]:
# image_path = './captures/SQUAT_pauses_2023-01-10T02:34:35.626182.jpg'
# video_path = './captures/SQUAT_pauses_2023-01-10T02:34:35.626182.m4v'

ex_data = execute_exercise_session(EXERCISE, name, ref_id, target_reps, image_path, video_path)
# pairs.append((ex_data, reference_data))

### Session 5: Half reps

In [18]:
date_str = datetime.now().isoformat()
name = f"{EXERCISE}_{'half_reps'}_{date_str}"
target_reps = 4

# Capture video recording and image
image_path, video_path = capture_session(name,10,12, base_dir='./captures/')

Entering Video Loop


In [19]:
ex_data = execute_exercise_session(EXERCISE, name, ref_id, target_reps, image_path, video_path)
# pairs.append((ex_data, reference_data))

Get data

In [20]:
# Get all exercise data 
ex_url = 'http://127.0.0.1:8000/api/v1/exercise_sessions/'
ref_url = 'http://127.0.0.1:8000/api/v1/reference_sessions/'

pp = pprint.PrettyPrinter(indent=4, sort_dicts=True)

r = requests.get(ex_url)

ex_sessions = r.json()

pairs = []

for ex_session in ex_sessions:
    id  = ex_session['reference']

    if id == 'de0e590d-e485-43ad-8034-24094d569345':
        r = requests.get(ref_url+ref_id)
        ref_data = r.json()
        pairs.append((ex_session, ref_data))

Write raw data to JSON file

In [21]:
date_str = datetime.now().isoformat()

path = os.path.join('./DTW_data_archive', f"{EXERCISE}_DTW_pairs_{date_str}.json")

with open(path, 'w') as f:
    json.dump(pairs, f, indent=4)

# Analyze Data

basically get a dataframe output

In [22]:
pp = pprint.PrettyPrinter(indent=4, sort_dicts=True)

for pair in pairs:
    ex_data, ref_data = pair
    pp.pprint(ex_data['name'])
    pp.pprint(ex_data['analytics']['DTW'])

'LEG_RAISE_base_session_2023-01-10T14:37:19.476495'
{   'joint_angles_agg': {   'accuracy': 0.9040023482071047,
                            'dist': 7879.48725916085,
                            'normalized_dist': 11.81332422662796,
                            'path_len': 513},
    'joint_angles_agg_joi': {   'accuracy': 0.8687139738435261,
                                'dist': 2709.743579869622,
                                'normalized_dist': 4.06258407776555,
                                'path_len': 516},
    'joint_angles_agg_norm': {   'accuracy': 0.9029807884890347,
                                 'dist': 8134.090693079331,
                                 'normalized_dist': 12.195038520358818,
                                 'path_len': 524},
    'joint_angles_agg_norm_joi': {   'accuracy': 0.8439787661164688,
                                     'dist': 3357.5769531735923,
                                     'normalized_dist': 5.033848505507635,
                       

In [2]:
import json

pp = pprint.PrettyPrinter(indent=4, sort_dicts=True)
# Opening JSON file
f = open('./DTW_data_archive/SQUAT_DTW_pairs_2023-01-10T03:17:53.997716.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

for pair in data:
    ex_data, ref_data = pair
    pp.pprint(ex_data['name'])
    pp.pprint(ex_data['analytics']['DTW'])


'SQUAT_base_session_2023-01-10T03:09:55.626388'
{   'joint_angles_agg': {   'accuracy': 0.8561540407251426,
                            'dist': 11622.753509408478,
                            'normalized_dist': 17.610232590012846,
                            'path_len': 505},
    'joint_angles_agg_joi': {   'accuracy': 0.8703210674761661,
                                'dist': 7889.666254750053,
                                'normalized_dist': 11.954039779924322,
                                'path_len': 507},
    'joint_angles_agg_norm': {   'accuracy': 0.8233796957245536,
                                 'dist': 15288.253538082641,
                                 'normalized_dist': 23.164020512246427,
                                 'path_len': 541},
    'joint_angles_agg_norm_joi': {   'accuracy': 0.8339969525965738,
                                     'dist': 10856.599300184074,
                                     'normalized_dist': 16.449392879066778,
                    

In [3]:
import json

pp = pprint.PrettyPrinter(indent=4, sort_dicts=True)
# Opening JSON file
f = open('./DTW_data_archive/LEG_RAISE_DTW_pairs_2023-01-10T14:53:52.291618.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

for pair in data:
    ex_data, ref_data = pair
    pp.pprint(ex_data['name'])
    pp.pprint(ex_data['analytics']['DTW'])

'LEG_RAISE_base_session_2023-01-10T14:37:19.476495'
{   'joint_angles_agg': {   'accuracy': 0.9040023482071047,
                            'dist': 7879.48725916085,
                            'normalized_dist': 11.81332422662796,
                            'path_len': 513},
    'joint_angles_agg_joi': {   'accuracy': 0.8687139738435261,
                                'dist': 2709.743579869622,
                                'normalized_dist': 4.06258407776555,
                                'path_len': 516},
    'joint_angles_agg_norm': {   'accuracy': 0.9029807884890347,
                                 'dist': 8134.090693079331,
                                 'normalized_dist': 12.195038520358818,
                                 'path_len': 524},
    'joint_angles_agg_norm_joi': {   'accuracy': 0.8439787661164688,
                                     'dist': 3357.5769531735923,
                                     'normalized_dist': 5.033848505507635,
                       